# [Brazil Olist] Ecommerce_Data
brazil olist 는 몇년도에 만들어졌는가 (더 찾아봐야함)
- 브라질의 Olist는 2015년에 설립되었습니다. Olist는 온라인 판매자들이 여러 전자상거래 플랫폼에서 상품을 판매할 수 있도록 지원하는 마켓플레이스 통합 서비스로, 특히 소규모 상점과 브랜드들이 아마존, Mercado Livre 등 대형 마켓플레이스에서 쉽게 판매할 수 있도록 돕습니다.


- 특히 Olist 같은 플랫폼 비즈니스 모델은 네트워크 효과를 극대화해야 하므로 초기에는 판매자와 구매자 양쪽을 모두 확장하는 데 집중하게 됩니다. 2017~2018년은 Olist가 고객 기반을 확장하고 다양한 마켓플레이스와의 통합을 강화하며 시장에서 자리를 잡아가던 시기였을 것입니다. 성숙기에 도달하는 데는 보통 5년 이상이 걸리기 때문에, 이 시기에는 아직 성장 중이었을 것으로 보입니다.

- about olist : https://valorcapitalgroup.com/case-studies/olist-redesigned-the-marketplace-business-model-to-fit-the-realities-of-ecommerce-in-brazil/

# 0.0) 데이터 전처리 
- 변수 추가

In [7]:
import pandas as pd
import numpy as np

customer_df = pd.read_csv('/Users/johaneul/Desktop/coding_test/data_analysis/olist/olist_customers_dataset.csv')
geo = pd.read_csv('/Users/johaneul/Desktop/coding_test/data_analysis/olist//olist_geolocation_dataset.csv')
order_items_df = pd.read_csv('/Users/johaneul/Desktop/coding_test/data_analysis/olist/olist_order_items_dataset.csv')
order_reviews_df = pd.read_csv('/Users/johaneul/Desktop/coding_test/data_analysis/olist/olist_order_reviews_dataset.csv')
order_payments_df = pd.read_csv('/Users/johaneul/Desktop/coding_test/data_analysis/olist/olist_order_payments_dataset.csv')
orders_df = pd.read_csv('/Users/johaneul/Desktop/coding_test/data_analysis/olist/olist_orders_dataset.csv')
products_df = pd.read_csv('/Users/johaneul/Desktop/coding_test/data_analysis/olist/olist_products_dataset.csv')
category_name_df = pd.read_csv('/Users/johaneul/Desktop/coding_test/data_analysis/olist/product_category_name_translation.csv')
sellers_df = pd.read_csv('/Users/johaneul/Desktop/coding_test/data_analysis/olist/olist_sellers_dataset.csv')


customer_df.columns =['고객id', '고객_유니크id', '고객_우편번호앞자리','고객_도시', '고객_주(state)'] 
orders_df.columns = ['주문id', '고객id', '주문상태', '구매시간', '주문승인일', '배송시작일', '배송일', '예상배송일']

order_items_df.columns =['주문id', '주문_항목id', '제품id', '판매자id',    '배송제한날짜', '제품가격', '운임가격'] 
order_payments_df.columns =['주문id', '결제방법수', '결제유형', '결제할부', '결제금액']  
order_reviews_df.columns =['리뷰id', '주문id', '평점', '리뷰제목', '리뷰내용', '리뷰일시', '리뷰답변일시'] 
products_df.columns =['제품id', '제품_카테고리', '제품_이름길이', '제품_설명길이', '제품_사진수', '제품_무게g', '제품_길이cm', '제품_높이cm', '제품_너비cm']  

geo.columns = ['우편번호 앞자리', '위도', '경도', '도시', '주']  

  
category_name_df.columns =['제품_카테고리_포르투갈어', '제품_카테고리_영어']  
sellers_df.columns =['판매자id', '판매자 우편번호 앞자리', '판매자 도시', '판매자 주(state)'] 

df = pd.merge(customer_df, orders_df, left_on = '고객id', right_on ='고객id')
df = pd.merge(df,order_items_df, left_on = '주문id', right_on ='주문id' )
df = pd.merge(df,order_payments_df, left_on = '주문id', right_on ='주문id' )
df = pd.merge(df,order_reviews_df, left_on = '주문id', right_on ='주문id' )
df = pd.merge(df,products_df, left_on = '제품id', right_on ='제품id' )
df = pd.merge(df,category_name_df, left_on = '제품_카테고리', right_on ='제품_카테고리_포르투갈어' )

df = pd.merge(df,sellers_df, left_on = '판매자id', right_on ='판매자id' )

df.drop(['고객id'],axis=1, inplace=True)

# 중복 행 제거
df.duplicated(subset = '주문id').value_counts()
df =  df[~df.duplicated(['주문id'])]
df['주문승인일']= df['주문승인일'].astype('str')
df['year'] = df['주문승인일'].str[:4]
df['year_month'] = df['주문승인일'].str[:7]
df['date'] = df['주문승인일'].str[:10]

df['date'] = pd.to_datetime(df['date']) 
df['quarter'] = df['date'].dt.to_period('Q') 
df['quarter']= df['quarter'].astype('str')

df = df[(df['year'] =='2017') | (df['year'] =='2018')]
df = df[df['year_month'] != '2018-09'] 
df.drop(['제품_카테고리','제품_카테고리_포르투갈어'],axis=1, inplace=True)  #'제품_카테고리_포르투갈어'

- 카테고리 번역

In [8]:
# 카테고리 영문 리스트
categories_english = list(df['제품_카테고리_영어'].unique())
# 한국어 번역 리스트
categories_korean = [

    '사무용 가구', '스포츠 및 레저', '가구 및 장식', '자동차', '유아용품', 
    '거실 가구', '주방, 다이닝, 세탁, 정원 가구', '건강 및 미용', 
    '가정용품', '컴퓨터 및 액세서리', '장난감', '정원 도구', '애완동물 상점', '시계 및 선물', 
    '향수', '문구류', '건축 도구 및 조명', '신기한 물건', '가전제품', 
    '신호 및 보안', '건축 도구 및 건축 자재', '오디오', '마켓 플레이스', 
    '전자제품', '전화기', '미술', '건축 도구 및 도구류', '악기', 
    '산업, 상업 및 비즈니스', '식품', '유선 전화', '농업 산업 및 상업', 
    '가방 및 액세서리', '침실 가구', '에어컨', '침대, 욕실, 테이블 용품', 
    '가전제품 2', '소형 가전제품', '소형 가전제품(오븐 및 커피)', 
    '가정용 건축 자재', '패션 가방 및 액세서리', '건축 도구 및 정원 도구', 
    '남성 패션 의류', '패션 신발', '패션 속옷 및 비치웨어', '콘솔 및 게임', 
    '가정용 편안함', '침대 매트리스 및 쿠션 가구', '스포츠 패션', '파티 용품', 
    '아동 패션 의류', '크리스마스 용품', '여성 패션 의류', 
    '건축 도구 및 안전 용품', '음료', '일반 서적', '영화 및 사진', '요리', 
    '식품 및 음료', '음악', '컴퓨터', '예술 및 공예', '전문 서적', '수입 서적', 
    '태블릿, 인쇄 및 이미지', '꽃', 'CD, DVD 및 뮤지컬', '보안 및 서비스', 
    'DVD 및 블루레이', '가정용 편안함 2', '기저귀 및 위생용품'
]

# 영어-한국어 딕셔너리 생성
category_dict = dict(zip(categories_english, categories_korean))

# DataFrame에 컬럼 추가 (pandas 활용 예시)
import pandas as pd

# 예시 데이터프레임
catcat = pd.DataFrame({
    'category': categories_english
})

# 한국어 번역 컬럼 추가
catcat['category_korean'] = catcat['category'].map(category_dict)

df = pd.merge(df, catcat, left_on = '제품_카테고리_영어', right_on = 'category')
df.drop( 'category', axis=1, inplace=True)

- 환율 적용해서 BRL -> (USD)로 변경

In [9]:
### 
## 브라질 환율 -> Usd 환율 변경 (2017 평균 환율 : 1 USD = 3.19 BRL, 2018은  3.65 BRL )

df['제품가격_usd'] = ''
df['결제금액_usd'] =''
df['운임가격_usd'] =''

ddd17 = df[df['year'] =='2017']
ddd18 = df[df['year'] =='2018']

ddd17['제품가격_usd'] = ddd17['제품가격'] / 3.19
ddd17['결제금액_usd'] = ddd17['결제금액'] / 3.19
ddd17['운임가격_usd'] = ddd17['운임가격'] / 3.19

ddd18['제품가격_usd'] = ddd18['제품가격'] / 3.65
ddd18['결제금액_usd'] = ddd18['결제금액'] / 3.65
ddd18['운임가격_usd'] = ddd18['운임가격'] / 3.65

df= pd.concat([ddd17, ddd18])

geo_Avg = geo.groupby('우편번호 앞자리').agg({
    '위도': 'mean',
    '경도': 'mean',
    '도시': 'first',  # 도시 정보는 첫 번째 값을 사용
    '주': 'first'     # 주 정보도 첫 번째 값을 사용
}).reset_index()


/var/folders/32/kf__h26s36l08d_ty7fh2xjr0000gn/T/ipykernel_2776/3661438180.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddd17['제품가격_usd'] = ddd17['제품가격'] / 3.19
/var/folders/32/kf__h26s36l08d_ty7fh2xjr0000gn/T/ipykernel_2776/3661438180.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddd17['결제금액_usd'] = ddd17['결제금액'] / 3.19
/var/folders/32/kf__h26s36l08d_ty7fh2xjr0000gn/T/ipykernel_2776/3661438180.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [10]:
sellers_df = pd.merge(sellers_df, geo_Avg, left_on = '판매자 우편번호 앞자리' , right_on = '우편번호 앞자리', how='left')
sellers_df.drop(['도시', '주','우편번호 앞자리'],axis=1,inplace=True)
sellers_df.columns =['판매자id', '판매자 우편번호 앞자리', '판매자 도시', '판매자 주(state)',  '위도_판매자',  '경도_판매자']

df= pd.merge(df, geo_Avg, left_on= '고객_우편번호앞자리', right_on= '우편번호 앞자리', how='left')

df.drop(['우편번호 앞자리', '도시','주'],axis=1, inplace=True)


df.columns = ['고객_유니크id', '고객_우편번호앞자리', '고객_도시', '고객_주(state)', '주문id', '주문상태',
       '구매시간', '주문승인일', '배송시작일', '배송일', '예상배송일', '주문_항목id', '제품id', '판매자id',
       '배송제한날짜', '제품가격', '운임가격', '결제방법수', '결제유형', '결제할부', '결제금액', '리뷰id', '평점',
       '리뷰제목', '리뷰내용', '리뷰일시', '리뷰답변일시', '제품_이름길이', '제품_설명길이', '제품_사진수',
       '제품_무게g', '제품_길이cm', '제품_높이cm', '제품_너비cm', '제품_카테고리_영어', '판매자 우편번호 앞자리',
       '판매자 도시', '판매자 주(state)', 'year', 'year_month', 'date', 'quarter',
       'category_korean', '제품가격_usd', '결제금액_usd', '운임가격_usd','위도_고객', '경도_고객']
 
df= pd.merge(df, sellers_df, left_on= '판매자id', right_on = '판매자id', how = 'left')
df.drop(['판매자 우편번호 앞자리_y', '판매자 도시_y', '판매자 주(state)_y',],axis=1, inplace=True)

df.columns = ['고객_유니크id', '고객_우편번호앞자리', '고객_도시', '고객_주(state)', '주문id', '주문상태',
       '구매시간', '주문승인일', '배송시작일', '배송일', '예상배송일', '주문_항목id', '제품id', '판매자id',
       '배송제한날짜', '제품가격', '운임가격', '결제방법수', '결제유형', '결제할부', '결제금액', '리뷰id', '평점',
       '리뷰제목', '리뷰내용', '리뷰일시', '리뷰답변일시', '제품_이름길이', '제품_설명길이', '제품_사진수',
       '제품_무게g', '제품_길이cm', '제품_높이cm', '제품_너비cm', '제품_카테고리_영어',
       '판매자 우편번호 앞자리', '판매자 도시', '판매자 주(state)', 'year', 'year_month',
       'date', 'quarter', 'category_korean', '제품가격_usd', '결제금액_usd',
       '운임가격_usd', '위도_고객', '경도_고객', '위도_판매자', '경도_판매자']

- 고객-판매자간 거리 계산 (파생변수 추가)

In [11]:
 
'''
고객과 판매자 간 거리 계산 - '하버사인 공식'사용
고객과 판매자 간의 거리를 계산하려면 위도와 경도를 기반으로 하버사인 공식을 사용할 수 있습니다. 
이 공식은 구면 좌표계에서 두 지점 간의 대원을 따라 계산된 거리를 구하는 데 유용합니다.

'''

import pandas as pd
import numpy as np

# 지구 반지름 (단위: km)
R = 6371.0

# 하버사인 공식을 사용하여 거리 계산 함수 정의
def haversine(lat1, lon1, lat2, lon2):
    # 위도와 경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # 위도, 경도의 차이 계산
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # 하버사인 공식 적용
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # 최종 거리 계산 (지구 반지름을 곱함)
    return R * c
 
# 거리 계산하여 새로운 열로 추가
df['고객_판매자_거리_km'] = df.apply(lambda row: haversine(row['위도_고객'], row['경도_고객'], row['위도_판매자'], row['경도_판매자']), axis=1)
 
d  = pd.DataFrame(df['고객_유니크id'].value_counts()).reset_index()
d.columns = ['고객_유니크id', '구매count']
df  = pd.merge(df, d , left_on = '고객_유니크id', right_on = '고객_유니크id', how = 'left')

df.drop(['고객_우편번호앞자리',  '위도_고객', '경도_고객', '위도_판매자',
       '경도_판매자','판매자 우편번호 앞자리','제품가격',  '운임가격',  '결제금액'],axis=1,inplace=True)
       
# 날짜 데이터를 datetime 형식으로 변환
df['배송일'] = pd.to_datetime(df['배송일'])
df['예상배송일'] = pd.to_datetime(df['예상배송일'])

df['구매시간'] = pd.to_datetime(df['구매시간'])
df['주문승인일'] = pd.to_datetime(df['주문승인일'])
df['배송시작일'] = pd.to_datetime(df['배송시작일'])
df['리뷰일시'] = pd.to_datetime(df['리뷰일시'])
df['리뷰답변일시'] = pd.to_datetime(df['리뷰답변일시'])
    
# 배송 지연시간 계산 (배송일 - 예상배송일) : 고객이 예상한 배송일과 실제 배송일 사이의 차이.
# 고객 관점: 예상된 날짜보다 늦은 배송은 큰 불만족 요인이 될 수 있으며, 신뢰도에 영향을 미칩니다.
df['배송지연시간'] = (df['배송일'] - df['예상배송일']).dt.days
# 주문 처리 시간: (주문승인일 - 구매시간)  : 구매 시점부터 주문이 승인되기까지 걸린 시간.
df['주문처리시간'] = (df['주문승인일'] - df['구매시간']).dt.days
#배송시간
df['배송소요시간'] = (df['배송일'] - df['배송시작일']).dt.days
# 전체 물류 처리 시간 
df['전체물류처리시간'] = (df['배송일'] - df['주문승인일']).dt.days
# 리뷰 답변 처리 속도
df['리뷰답변_처리속도'] = (df['리뷰답변일시'] - df['리뷰일시']).dt.days

In [12]:
# 카테고리 일부 병합 
df['category_korean'] = df['category_korean'].str.replace('가정용 편안함 2', '가정용 편안함')
df['category_korean']= df['category_korean'].str.replace('가전제품 2', '가전제품')
df['category_korean'] = df['category_korean'].str.replace('소형 가전제품(오븐 및 커피)', '소형 가전제품') 

df['category_korean'] = df['category_korean'].str.replace( '건축 도구 및 건축 자재',  '건축 도구 및 자재') 
df['category_korean'] = df['category_korean'].str.replace('건축 도구 및 안전 용품', '건축 도구 및 자재') 

df['category_korean'] = df['category_korean'].str.replace( '건축 도구 및 도구류', '건축 도구 및 자재') 
df['category_korean'] = df['category_korean'].str.replace(   '건축 도구 및 조명', '건축 도구 및 자재') 
df['category_korean'] = df['category_korean'].str.replace( '건축 도구 및 정원 도구', '건축 도구 및 자재') 

df['category_korean'] = df['category_korean'].str.replace('패션 가방 및 액세서리', '가방 및 액세서리') 

# 대분류 컬럼 만들기
df['category_대분류']=''
# 카테고리 대분류 매핑
category_mapping = {
    '가전/전자제품': [ '컴퓨터 및 액세서리','에어컨', '컴퓨터', 'DVD 및 블루레이', '가전제품', '소형 가전제품', '전자제품', '오디오',   '전화기', '유선 전화', '태블릿, 인쇄 및 이미지'],
    '가구/인테리어': ['꽃', '가정용 건축 자재', '건축 도구 및 자재','사무용 가구', '거실 가구', '침실 가구', '침대, 욕실, 테이블 용품', '가구 및 장식', '정원 도구', '주방, 다이닝, 세탁, 정원 가구', '침대 매트리스 및 쿠션 가구'],
    '패션/액세서리': ['남성 패션 의류', '여성 패션 의류', '아동 패션 의류', '가방 및 액세서리', '패션 신발', '패션 속옷 및 비치웨어', '스포츠 패션', '시계 및 선물'],
    '취미/레저/스포츠': ['스포츠 및 레저','자동차', '악기', '파티 용품', '콘솔 및 게임','CD, DVD 및 뮤지컬', '크리스마스 용품', '영화 및 사진', '음악'],
    '건강/미용': ['건강 및 미용', '향수', '기저귀 및 위생용품'],
    '식품/음료': ['식품', '식품 및 음료', '음료', '요리'],
    '가정용품/생활용품': ['가정용품',  '가정용 편안함', '신호 및 보안', '신기한 물건', ],
    '애완/유아': ['유아용품', '장난감', '애완동물 상점', ],
    '서적/미술/교육': ['일반 서적', '전문 서적', '수입 서적', '미술', '예술 및 공예', '문구류'],
    '비즈니스/산업': ['산업, 상업 및 비즈니스', '농업 산업 및 상업', '보안 및 서비스', '마켓 플레이스'] 
}


# 카테고리 매핑 함수 정의
def map_to_category(category):
    for key, values in category_mapping.items():
        if category in values:
            return key
    return '기타'  # 대분류에 속하지 않는 항목은 '기타'로 처리

# 대분류 컬럼 추가
df['category_대분류'] = df['category_korean'].apply(map_to_category)

In [13]:
df.columns

Index(['고객_유니크id', '고객_도시', '고객_주(state)', '주문id', '주문상태', '구매시간', '주문승인일',
       '배송시작일', '배송일', '예상배송일', '주문_항목id', '제품id', '판매자id', '배송제한날짜', '결제방법수',
       '결제유형', '결제할부', '리뷰id', '평점', '리뷰제목', '리뷰내용', '리뷰일시', '리뷰답변일시',
       '제품_이름길이', '제품_설명길이', '제품_사진수', '제품_무게g', '제품_길이cm', '제품_높이cm',
       '제품_너비cm', '제품_카테고리_영어', '판매자 도시', '판매자 주(state)', 'year', 'year_month',
       'date', 'quarter', 'category_korean', '제품가격_usd', '결제금액_usd',
       '운임가격_usd', '고객_판매자_거리_km', '구매count', '배송지연시간', '주문처리시간', '배송소요시간',
       '전체물류처리시간', '리뷰답변_처리속도', 'category_대분류'],
      dtype='object')

In [14]:
df

,고객_유니크id,고객_도시,고객_주(state),주문id,주문상태,구매시간,주문승인일,배송시작일,배송일,예상배송일,...,결제금액_usd,운임가격_usd,고객_판매자_거리_km,구매count,배송지연시간,주문처리시간,배송소요시간,전체물류처리시간,리뷰답변_처리속도,category_대분류
0,861eff4711a542e4b93843c6dd7febb0,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,...,46.040752,6.858934,348.770656,1,-11.0,0,1.0,8.0,4,가구/인테리어
1,9eae34bbd3a474ec5d07949ca7de67c0,santarem,PA,c1d2b34febe9cd269e378117d6681172,delivered,2017-11-09 00:50:13,2017-11-10 00:47:48,2017-11-22 01:43:37,2017-11-28 00:09:50,2017-12-19,...,86.454545,7.805643,2506.927836,1,-21.0,0,5.0,17.0,2,가구/인테리어
2,3c799d181c34d51f6d44bbbc563024db,nova santa rita,RS,b1a5d5365d330d10485e0203d54ab9e8,delivered,2017-05-07 20:11:26,2017-05-08 22:22:56,2017-05-19 20:16:31,2017-05-26 09:54:04,2017-06-12,...,44.078370,4.896552,860.018003,1,-17.0,1,6.0,17.0,1,가구/인테리어
3,567ab47ca4deb92d46dbf54dce07d0a7,angelina,SC,574fe1739f65af76badd0999db300b4f,delivered,2017-03-23 15:10:17,2017-03-23 15:25:11,2017-03-28 18:23:51,2017-04-11 10:16:56,2017-04-24,...,44.529781,4.721003,525.701996,1,-13.0,0,13.0,18.0,3,가구/인테리어
4,f40ab89b622248b7ca125af4b486b887,contagem,MG,e0b26f14d2bcc710bb02f77a4628763b,delivered,2017-05-16 10:00:49,2017-05-17 03:45:27,2017-05-23 10:35:48,2017-05-29 12:04:19,2017-06-07,...,48.808777,9.626959,461.801509,1,-9.0,0,6.0,12.0,1,가구/인테리어
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96193,89be58cbdd6ef318e3ed93fdb22be178,itapeva,SP,048297b4e795502db60be292e12dd390,delivered,2018-03-26 03:50:48,2018-03-26 04:08:31,2018-03-29 16:22:03,2018-04-09 18:46:51,2018-04-12,...,53.027397,3.756164,214.865163,2,-3.0,0,11.0,14.0,9,건강/미용
96194,a2ba94a4e9d0ab6b5942c6210bd5d16e,bonfim paulista,SP,fa612a1874826daa30fed5e3073b9d0d,delivered,2018-02-23 07:23:09,2018-02-23 07:35:30,2018-02-27 15:13:08,2018-03-01 16:22:31,2018-03-15,...,41.304110,3.476712,273.494624,1,-14.0,0,2.0,6.0,3,건강/미용
96195,c6c805ba1449708b259b6da9375318ec,jundiai,SP,6bcce39be127575854779e39a50f2059,delivered,2018-04-28 02:23:24,2018-05-01 05:32:22,2018-05-04 11:35:00,2018-05-08 00:18:58,2018-05-17,...,15.383562,3.054795,39.880966,1,-9.0,3,3.0,6.0,2,건강/미용
96196,7e3b6916cdc9629811177f17f7ab07ce,sao paulo,SP,796033ffde6ba06e0abe387e36ee8fe0,delivered,2018-08-26 14:17:08,2018-08-26 14:25:15,2018-08-27 07:45:00,2018-08-29 18:48:44,2018-09-11,...,43.553425,5.227397,438.217834,1,-13.0,0,2.0,3.0,2,건강/미용


# 0.1) 유저 리뷰 번역

- 리뷰 : 포루투갈어 -> 한국어

    - 평점대 별로 어떤 리뷰?? (유저 pain point 찾기)

    - 재구매 고객은 주로 어떤 리뷰 내용을 남기나??


    - 제품 별로 시간대... 점점 어떤 리뷰를 남기는지.. 점점 평점이 낮아지거나 하면 (배송이슈 등으로 ) 이런것도 뭔가 문제 해결해야할 부분일테고

In [ ]:

## 리뷰 번역, 나머지 ? 
rev_lego_17 = df[(df['year'] =='2017') &( df['구매count']==1)]
rev_lego_17 = rev_lego_17[['주문id', '리뷰내용','평점', '구매count']].dropna()
rev_lego_17['리뷰내용'] = rev_lego_17['리뷰내용'].str.replace('\r\n', '')  
 
rev_lego_17_2 = df[(df['year'] =='2017') &( df['구매count']>1)]
rev_lego_17_2 = rev_lego_17_2[['주문id', '리뷰내용','평점', '구매count']].dropna()
rev_lego_17_2['리뷰내용'] = rev_lego_17_2['리뷰내용'].str.replace('\r\n', '')  

In [226]:
rev_lego_18 = df[(df['year'] =='2018') &( df['구매count']==1)]
rev_lego_18 = rev_lego_18[['주문id', '리뷰내용','평점', '구매count']].dropna()
rev_lego_18['리뷰내용'] = rev_lego_18['리뷰내용'].str.replace('\r\n', '')  

rev_lego_18_2 = df[(df['year'] =='2018') &( df['구매count']>1)]
rev_lego_18_2 = rev_lego_18_2[['주문id', '리뷰내용','평점', '구매count']].dropna()
rev_lego_18_2['리뷰내용'] = rev_lego_18_2['리뷰내용'].str.replace('\r\n', '')  

In [227]:
print(len(rev_lego_18))

20239


In [228]:
print(len(rev_lego_18_2))

1149


In [204]:
 

# 필요한 라이브러리 설치 (설치가 안 되어 있는 경우)
# pip install googletrans==4.0.0-rc1
import pandas as pd
from googletrans import Translator

# 번역기 객체 생성
translator = Translator()

# 포르투갈어 문장을 한국어로 번역
text_to_translate =  "Olá, como você está?"
translated_text = translator.translate(text_to_translate, src='pt', dest='ko')

print("원본 문장 (포르투갈어):", text_to_translate)
print("번역된 문장 (한국어):", translated_text.text)

원본 문장 (포르투갈어): Olá, como você está?
번역된 문장 (한국어): 안녕하세요. 어떻게 지내세요?


In [207]:

'''
rev_lego_17 = df[(df['year'] =='2017') &( df['구매count']==1)]
rev_lego_17 = rev_lego_17[['주문id', '리뷰내용','평점', '구매count']].dropna()
rev_lego_17['리뷰내용'] = rev_lego_17['리뷰내용'].str.replace('\r\n', '')  
'''
rev_lego_17_2 = df[(df['year'] =='2017') &( df['구매count']>1)]
rev_lego_17_2 = rev_lego_17_2[['주문id', '리뷰내용','평점', '구매count']].dropna()
rev_lego_17_2['리뷰내용'] = rev_lego_17_2['리뷰내용'].str.replace('\r\n', '')  

In [208]:
rev_lego_17_2 = rev_lego_17_2.reset_index(drop=True)

In [209]:
rev_lego_17_2

,주문id,리뷰내용,평점,구매count
0,2fc9faf8c6b3ffe951029b8bae263bb6,Mandei uma reclamação para a Americanad também...,1,2
1,21862ad09cda6a079859cb1fb77d5da2,"Produto de péssima qualidade, as 3 partes de m...",1,2
2,d4d140e1cbc20856440a8343432848c4,Ainda não recebi minha compra. Já passou do pr...,1,2
3,d708b88543d7c2b7c9eb0b31864c8ca1,"O produto chegou bem antes do prazo, porém uma...",4,2
4,ef118656ef537afb61840ff77a86a067,"Entrega rápida, aconteceu em menos de 7 dias! ...",5,2
...,...,...,...,...
1217,094303c767dff093564257cc4d0e2c29,vou continuar comprando aqui.,5,2
1218,c40ef1f09f4ba772a63865df0274f6b8,Produto chegou conforme combinado,5,2
1219,aae35f8d7c5a9806060218e572d8370d,Produto correto e entrega no prazo.,4,2
1220,1a4fe7b4e0810e3c234a4e39775b6d53,O produto é bom e muto bem feito,5,2


In [215]:
# rev_lego_17 = rev_lego_17[856:].reset_index(drop=True) 

rev_lego_17_2= rev_lego_17_2[668:].reset_index(drop=True) 


In [217]:
from tqdm import tqdm 
from googletrans import Translator
 
translator = Translator() 

rev_lego_17_2['리뷰내용_한국어번역'] = ''  

for i in tqdm(range(len(rev_lego_17_2))):  
        try:
                rev_lego_17_2['리뷰내용_한국어번역'].iloc[i] = translator.translate(rev_lego_17_2['리뷰내용'].iloc[i] , src='pt', dest='ko').text
        except TypeError:
                pass
 

  0%|          | 0/554 [00:00<?, ?it/s]/var/folders/32/kf__h26s36l08d_ty7fh2xjr0000gn/T/ipykernel_46263/2773930357.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_lego_17_2['리뷰내용_한국어번역'].iloc[i] = translator.translate(rev_lego_17_2['리뷰내용'].iloc[i] , src='pt', dest='ko').text
  0%|          | 1/554 [00:00<05:19,  1.73it/s]/var/folders/32/kf__h26s36l08d_ty7fh2xjr0000gn/T/ipykernel_46263/2773930357.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_lego_17_2['리뷰내용_한국어번역'].iloc[i] = translator.translate(rev_lego_17_2['리뷰내용'].iloc[i] , src='pt', dest='ko').text
  0%|          | 2/554 [00:02<10:49,  1.18s/it]/

In [211]:
# df_revvv2 = pd.DataFrame() 
df_revvv2 = pd.concat([df_revvv2, rev_lego_17_2  ])

In [218]:
# df_revvv = pd.concat([df_revvv, rev_lego_17 ])
# df_revvv.to_csv('리뷰번역_olist_ecommerce_1번구매고객_2017.csv') 
# df_revvv2.to_csv('리뷰번역_olist_ecommerce_재구매고객_2017.csv')

In [222]:
df_revvv2 = pd.read_csv('리뷰번역_olist_ecommerce_재구매고객_2017.csv', index_col=0)
df_revvv2

,주문id,리뷰내용,평점,구매count,리뷰내용_한국어번역
0,2fc9faf8c6b3ffe951029b8bae263bb6,Mandei uma reclamação para a Americanad também...,1,2,나는 Americanonad에게 불만을 보냈다. 그들은 나를 돌려주지 않았다.크롬 ...
1,21862ad09cda6a079859cb1fb77d5da2,"Produto de péssima qualidade, as 3 partes de m...",1,2,"품질이 좋지 않은 제품, 나무의 3 개 부분이 긁히고 부서졌습니다.이셀의 끔찍한 그..."
2,d4d140e1cbc20856440a8343432848c4,Ainda não recebi minha compra. Já passou do pr...,1,2,아직 구매를받지 못했습니다.예상 마감일이 넘습니다!
3,d708b88543d7c2b7c9eb0b31864c8ca1,"O produto chegou bem antes do prazo, porém uma...",4,2,제품은 마감일 전에 잘 도착했지만 의자 중 하나가 열려 있었고 두 개의 나사가 손실...
4,ef118656ef537afb61840ff77a86a067,"Entrega rápida, aconteceu em menos de 7 dias! ...",5,2,"빠른 배달, 7 일 이내에 일어났습니다!훌륭한 품질과 인상적인 가격의 제품 !!나는..."
...,...,...,...,...,...
549,094303c767dff093564257cc4d0e2c29,vou continuar comprando aqui.,5,2,나는 여기서 계속 살 것이다.
550,c40ef1f09f4ba772a63865df0274f6b8,Produto chegou conforme combinado,5,2,합의 된대로 제품이 도착했습니다
551,aae35f8d7c5a9806060218e572d8370d,Produto correto e entrega no prazo.,4,2,제 시간에 올바른 제품 및 배송.
552,1a4fe7b4e0810e3c234a4e39775b6d53,O produto é bom e muto bem feito,5,2,제품은 훌륭하고 상호 잘 완성되었습니다


In [163]:
df_revvv = pd.read_csv('리뷰번역_olist_ecommerce_1번구매고객_2017.csv', index_col=0)
df_revvv

,주문id,리뷰내용,평점,구매count,리뷰내용_한국어번역
0,c1d2b34febe9cd269e378117d6681172,"GOSTARIA DE UMA SOLUÇÃO, ESTOU PRECISANDO MUIT...",1,1,솔루션을 원합니다. 제품이 많이 필요합니다.
1,b1a5d5365d330d10485e0203d54ab9e8,"Produto compatível com seu valor, muito bonito...",3,1,제품은 매우 아름답고 저렴하며 단순하지만 좋은 비용 혜택과 호환됩니다.
2,e0b26f14d2bcc710bb02f77a4628763b,Os encaixes para o encosto da cadeira estavam ...,4,1,의자 뒤쪽의 피팅은 잘못 정렬되었습니다.그는 적합하게 작업하고 마침내 제품을 조립했...
3,eaae5bd20fb15d85aa673d9b7c0e8ca5,Bom dia! Estou insatisfeita com a segunda comp...,2,1,좋은 아침이에요!2017 년 4 월 27 일까지 배달되기 때문에 두 번째 구매에 불...
4,0d831c5aa932379c1cf5b0b1b40b214b,A entrega foi no prazo e a venda ocorreu muito...,3,1,배달은 제 시간에 있었고 판매는 매우 잘 이루어졌습니다.원하는 것을 남긴이 제품의 품질.
...,...,...,...,...,...
381,fd0fb35d5a13ce62b474dfce15334b5f,A lannister está de parabéns. O produto chegou...,5,1,Lannister는 축하를 받아야합니다.제품은 마감일과 제품을 완벽한 상태로 도착하...
382,4a90fdde0d8f47bc83f32f3f39082f94,"Entrega rápida, produto igual ao anunciado.",5,1,"신속한 배송, 발표 된 제품."
383,4054f3a02c9191a518601fba04f06742,Veio bem antes do prazo e exatamente como na foto,5,1,마감일 이전에 그리고 정확히 사진과 마찬가지로 잘 왔습니다.
384,73cb429669ebb88bba7787201730c88e,Até agora não recebi,4,1,지금까지 나는받지 못했습니다
